# Exploring Database

In this notebook, we'll get acquainted with our Curriculum Lattes database by answering some initial questions about it. 

-----

**First of all, let's import a few things...**

In [1]:
import json
import pandas as pd
from pymongo import MongoClient

**Creating a Mongo Client...**

In [2]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['pbmnet']
curriculos = db.curriculos

## O que eu já tenho

- Nº Artigos
- Nº Pesquisadores
